In [ ]:
import ramanchada2.protocols.calib_ne_si_argmin2d_iter_gg as calgg
import ramanchada2 as rc2
import numpy as np
import matplotlib.pyplot as plt
import ramanchada2.misc.constants as rc2const

In [ ]:
# Load spectra
ne = rc2.spectrum.from_test_spe(OP=['01'], laser_wl=['785'], device=['Horiba'], provider=['FNMT'], sample=['Neon'])
si = rc2.spectrum.from_test_spe(OP=['01'], laser_wl=['785'], device=['Horiba'], provider=['FNMT'], sample=['S0N'])
ps = rc2.spectrum.from_test_spe(OP=['01'], laser_wl=['785'], device=['Horiba'], provider=['FNMT'], sample=['PST'])
wl = 785

# Perform all-in-one calibration
nesi_calib_fn = calgg.neon_silicon_calibration(ne, si, wl)

# Apply the calibration to Polystyrene
ps_cal = ps.scale_xaxis_fun(nesi_calib_fn)

# Plot Polystyrene
fig, ax = plt.subplots(figsize=(30, 8))
ps.plot(ax=ax, label='original')
ps_cal.plot(ax=ax, label='calibrated')
ax.set_title(f'Polystyrene')
ax.set_xlabel('1/cm')
twx = ax.twinx()
rc2.spectrum.from_delta_lines(rc2const.PST_RS_dict).plot(ax=twx, fmt='r:', label='reference')
twx.legend(loc='upper right')
ax.legend(loc='upper left')
fig.tight_layout()

In [ ]:
opts = [
    dict(OP=['01'], laser_wl=['532'], device=['BWtek'], provider=['FNMT']),
    dict(OP=['01'], laser_wl=['785'], device=['Horiba'], provider=['FNMT']),
    dict(OP=['03'], laser_wl=['785'], device=['Horiba'], provider=['FNMT']),
    dict(OP=['050'], laser_wl=['532'], device=['BWtek'], provider=['ICV']),
    dict(OP=['100'], laser_wl=['532'], device=['BWtek'], provider=['ICV']),
    dict(OP=['100'], laser_wl=['785'], device=['BWtek'], provider=['ICV']),
    dict(OP=['010'], laser_wl=['633'], device=['Horiba'], provider=['TOP']),
]
for opt in opts:
    base = str(opt).replace(':', '').replace(' ','')
    wl = int(opt['laser_wl'][0])

    # load spectra
    ne = rc2.spectrum.from_test_spe(**opt, sample=['Neon'])
    si = rc2.spectrum.from_test_spe(**opt, sample=['S0N'])
    ps = rc2.spectrum.from_test_spe(**opt, sample=['PST'])

    # Calculate wavelength calibration curve
    ne_calib_fn = calgg.neon_calibration(ne, wl)
    ne_cal = ne.scale_xaxis_fun(ne_calib_fn).dropna()

    # Calculate real laser wavelength
    si_nm = si.scale_xaxis_fun(ne_calib_fn).dropna()
    si_calib_fn, wl_real = calgg.silicon_calibration(si_nm, wl)
    si_cal = si_nm.scale_xaxis_fun(si_calib_fn)

    # All-in-one calibration
    nesi_calib_fn = calgg.neon_silicon_calibration(ne, si, wl)
    
    # Apply the calibration to Polystyrene
    ps_cal = ps.scale_xaxis_fun(nesi_calib_fn)

    # Plot Neon
    fig, ax = plt.subplots(figsize=(30, 8))
    ne.shift_cm_1_to_abs_nm_filter(wl).plot(ax=ax, label='original')
    ne_cal.plot(ax=ax, label='calibrated')
    twx = ax.twinx()
    rc2const.neon_wl_spe[wl].plot(ax=twx, fmt='g:', label='reference')
    ax.set_title(f'Neon {base} {wl}nm')
    ax.set_xlabel('nm')
    twx.legend(loc='upper right')
    ax.legend(loc='upper left')
    fig.tight_layout()
    #fig.savefig(f'{base}_{wl}nm_neon.pdf')
    display(fig)
    plt.close(fig)

    # Plot Silicon
    fig, ax = plt.subplots(figsize=(12, 5))
    si.plot(ax=ax, label='original')
    si_cal.plot(ax=ax, label='calibrated')
    ax.set_title(rf'Silicon {base} {wl}nm, $\lambda_{{laser}}={wl_real:8.3f}$nm')
    ax.set_xlabel('1/cm')
    ax.set_xlim(500, 540)
    ax.axvline(520.45)
    twx.legend(loc='upper right')
    ax.legend(loc='upper left')
    fig.tight_layout()
    #fig.savefig(f'{base}_{wl}nm_silicon.pdf')
    display(fig)
    plt.close(fig)
  
    # Plot Polystyrene
    fig, ax = plt.subplots(figsize=(30, 8))
    ps.plot(ax=ax, label='original')
    ps_cal.plot(ax=ax, label='calibrated')
    ax.set_title(f'Polystyrene {base} {wl}nm')
    ax.set_xlabel('1/cm')
    twx = ax.twinx()
    rc2.spectrum.from_delta_lines(rc2const.PST_RS_dict).plot(ax=twx, fmt='r:', label='reference')
    twx.legend(loc='upper right')
    ax.legend(loc='upper left')
    fig.tight_layout()
    #fig.savefig(f'{base}_{wl}nm_polystyrene.pdf')
    display(fig)
    plt.close(fig)